In [95]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

In [130]:
def lagrange_newton(f, df, phi, dphi, x_0, mu_0, epsilon, kmax):
    x = [x_0]
    mu = [mu_0]
    
    while np.linalg.norm(phi(x[-1], mu[-1])) > epsilon:
        
        dphi_mat = dphi(x[-1], mu[-1])
        phi_vec = -phi(x[-1], mu[-1])

        dres = np.linalg.solve(dphi(x[-1], mu[-1]).transpose(), -phi(x[-1], mu[-1]))

        x.append(x[-1] + dres[:len(x_0)])
        mu.append(mu[-1] + dres[len(x_0):])

    return np.array(x), np.array(mu)

In [131]:
def calculate_derivatives(f, h, variables): 
    mu = []
    if len(h) == 1:
        mu = [sp.Symbol("mu_0")]
    else:
        mu = list(sp.symbols(" ".join(f"mu{i}" for i in range(len(h)))))

    df = [f.diff(var) for var in variables]
    dh = [[h_i.diff(var) for h_i in h] for var in variables]
    phi = [df_i + sum(mu[j]*dh_i[j] for j in range(len(h))) for df_i, dh_i in zip(df, dh)] + h
    dphi = [[phi_i.diff(var) for phi_i in phi] for var in variables + mu]

    h_temp = [sp.lambdify(variables + mu, h_i) for h_i in h]
    f_temp = sp.lambdify(variables, f)

    dphi = [[sp.lambdify(variables + mu, phi_i.diff(var)) for phi_i in phi] for var in variables + mu]
    phi = [sp.lambdify(variables + mu, df_i + sum(mu[j]*dh_i[j] for j in range(len(h)))) for df_i, dh_i in zip(df, dh)] + h_temp
    df = [sp.lambdify(variables, f.diff(var)) for var in variables]

    dphi_lambda = lambda x, mu: np.array([[dphi_ij(*x, *mu) for dphi_ij in row] for row in dphi])
    phi_lambda = lambda x, mu: np.array([phi_i(*x, *mu) for phi_i in phi])
    df_lambda = lambda x: np.array([df_i(*x) for df_i in df])
    f_lambda = lambda x: f_temp(*x)
   
    return f_lambda, df_lambda, phi_lambda, dphi_lambda


In [132]:
x1, x2 = sp.symbols("x1 x2")

variables = [x1, x2]

f = 2*x1**4 - x2**4 + x1**2 -x1*x2 + 6*x2**2

h1 = 2*x1 - x2 + 4

x_0 = [0,0]
mu_0 = [0]

h = [h1]

epsilon = 10**(-3)
kmax = 200


In [133]:
f_l, df_l, phi_l, dphi_l = calculate_derivatives(f, h, variables)

x, mu = lagrange_newton(f_l, df_l, phi_l, dphi_l, x_0, mu_0, epsilon, kmax)

In [141]:
print("length of sequence:",len(x))
i = 0
for x_k, mu_k in zip(x,mu):
    print( f"x{i}:", x_k)
    print("   h:",h1.subs({x1: x_k[0], x2 : x_k[1]}).evalf(), "   norm(phi):", np.linalg.norm(phi_l(x_k, mu_k)))
    i+=1


length of sequence: 11
x0: [0. 0.]
   h: 4.00000000000000 norm(phi): 4.0
x1: [-2.  0.]
   h: 0 norm(phi): 64.0
x2: [-1.54929577  0.90140845]
   h: -4.44089209850063e-16 norm(phi): 9.48197588119801
x3: [-1.31901561  1.36196878]
   h: -4.44089209850063e-16 norm(phi): 3.274527587874575
x4: [-6.96652545 -9.93305091]
   h: 1.77635683940025e-15 norm(phi): 4420.391024319058
x5: [-5.57450152 -7.14900304]
   h: 0 norm(phi): 890.4873139198312
x6: [-4.74824796 -5.49649591]
   h: 0 norm(phi): 232.998854477657
x7: [-4.3477941 -4.6955882]
   h: 0 norm(phi): 43.99808606755286
x8: [-4.2354753  -4.47095061]
   h: 8.88178419700125e-16 norm(phi): 3.0874307406669033
x9: [-4.22659024 -4.45318048]
   h: -8.88178419700125e-16 norm(phi): 0.018723669182243996
x10: [-4.22653613 -4.45307225]
   h: -8.88178419700125e-16 norm(phi): 6.928219753159199e-07


In [127]:
x1, x2, x3 = sp.symbols("x1 x2 x3")

variables = [x1, x2, x3]

g = 1000 - x1**2 -2*x2**2 -x3**2 -x1*x2 - x1*x3

h1 = x1**2 + x2**2 + x3**2 - 25
h2 = 8*x1 + 14*x2 + 7*x3 -56

h = [h1,h2]

x_0 = [3,0.2,3]
mu_0 = [0,0]

epsilon = 10**(-5)
kmax = 200

In [128]:
g_l, dg_l, phi_l, dphi_l = calculate_derivatives(g, h, variables)

x, mu = lagrange_newton(g_l, dg_l, phi_l, dphi_l, x_0, mu_0, epsilon, kmax)

In [129]:
print(len(x))
print(x)
for x_k, mu_k in zip(x,mu):
    print(g_l(x_k))
    print()
    print(h1.subs({x1: x_k[0], x2 : x_k[1], x3 : x_k[2]}))

5
[[3.         0.2        3.        ]
 [3.63321358 0.15912215 3.52951161]
 [3.53117292 0.21453964 3.53529453]
 [3.51225635 0.21693024 3.55213227]
 [3.51212134 0.21698794 3.55217116]]
972.32
-6.96000000000000
960.8900728938873
0.683012981555404
961.6991430444024
0.0135168367635394
961.7143804889496
0.000647061336941590
961.7151721018298
2.30675745171993e-8
